# Interactive Acoustic Levitation Simulator

**Explore Flower of Life geometry with interactive controls!**

Use sliders to adjust:
- Number of emitters (7, 19, 37)
- Frequency (20-60 kHz)
- Ring radius (1-5λ)
- Particle size (1-8mm)
- Array geometry type

See results update in real-time!

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

# Import simulation modules
import sys
sys.path.append('../')

print("✓ Interactive simulator loaded!")
print("Use sliders below to explore parameter space.")

## 📊 Interactive Potential Field Visualization

In [ ]:
# Physical constants
SPEED_OF_SOUND = 343.0
AIR_DENSITY = 1.225

def calculate_field(frequency_khz, ring_radius_lambda, geometry_type, particle_size_mm):
    """
    Calculate and visualize acoustic potential field
    
    Args:
        frequency_khz: Carrier frequency (20-60 kHz)
        ring_radius_lambda: Ring radius in wavelengths (1-5)
        geometry_type: Array geometry ('fol', 'square', 'hexagonal')
        particle_size_mm: Particle diameter (1-8 mm)
    """
    
    # Calculate wavelength
    freq = frequency_khz * 1000
    wavelength = SPEED_OF_SOUND / freq
    k = 2 * np.pi / wavelength
    
    # Generate emitter positions based on geometry
    r1 = ring_radius_lambda * wavelength
    
    if geometry_type == 'fol':
        positions = [(0, 0, 0)]
        for i in range(6):
            theta = i * np.pi / 3
            positions.append((r1 * np.cos(theta), r1 * np.sin(theta), 0))
    
    elif geometry_type == 'square':
        positions = [(0, 0, 0)]
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                if len(positions) < 7:
                    positions.append((dx*r1, dy*r1, 0))
    
    elif geometry_type == 'hexagonal':
        r_uniform = 2.0 * wavelength  # Uniform spacing
        positions = [(0, 0, 0)]
        for i in range(6):
            theta = i * np.pi / 3
            positions.append((r_uniform * np.cos(theta), r_uniform * np.sin(theta), 0))
    
    positions = np.array(positions)
    
    # Calculate field on grid
    x_range = np.linspace(-0.04, 0.04, 60)
    y_range = np.linspace(-0.04, 0.04, 60)
    X, Y = np.meshgrid(x_range, y_range)
    z = 0.005  # 5mm above array
    
    # Particle parameters
    particle_radius = (particle_size_mm / 1000) / 2
    V0 = (4/3) * np.pi * particle_radius**3
    particle_density = 84  # Foam
    f1 = 1 - (AIR_DENSITY / particle_density)
    
    U = np.zeros_like(X)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # Acoustic pressure
            p_total = 0
            for ex, ey, ez in positions:
                r = np.sqrt((X[i,j] - ex)**2 + (Y[i,j] - ey)**2 + (z - ez)**2)
                if r < 1e-6:
                    r = 1e-6
                p_total += (1000 / r) * np.exp(1j * k * r)
            
            # Gor'kov potential
            p_mag_sq = np.abs(p_total)**2
            U[i,j] = -V0 * (f1 / (2 * AIR_DENSITY * SPEED_OF_SOUND**2)) * p_mag_sq
    
    # Visualization
    fig, ax = plt.subplots(figsize=(10, 8))
    
    im = ax.imshow(U*1e6, extent=[-40, 40, -40, 40], origin='lower',
                   cmap='RdYlBu_r', aspect='equal', interpolation='bilinear')
    
    # Plot emitters
    ax.scatter(positions[:,0]*1000, positions[:,1]*1000,
              c='black', s=200, marker='o', edgecolors='white', linewidths=2.5,
              label=f'{len(positions)} emitters', zorder=10)
    
    # Trap center
    ax.plot(0, 0, 'w+', markersize=25, markeredgewidth=4, zorder=15)
    
    ax.set_xlabel('X Position (mm)', fontweight='bold', fontsize=12)
    ax.set_ylabel('Y Position (mm)', fontweight='bold', fontsize=12)
    ax.set_title(f'{geometry_type.upper()} Array @ {frequency_khz:.1f} kHz | '
                f'Particle: {particle_size_mm:.1f}mm', 
                fontweight='bold', fontsize=14)
    ax.grid(True, alpha=0.2)
    ax.legend(loc='upper right', fontsize=11)
    
    plt.colorbar(im, ax=ax, label='Potential U (μJ)', fraction=0.046)
    
    # Metrics
    U_min = np.min(U)
    U_max = np.max(U)
    well_depth = (U_max - U_min) * 1e6
    
    annotation = f"Wavelength: {wavelength*1000:.2f} mm\n"
    annotation += f"Ring radius: {r1*1000:.1f} mm\n"
    annotation += f"Well depth: {well_depth:.1f} μJ"
    
    ax.text(0.05, 0.95, annotation,
           transform=ax.transAxes, fontsize=11, fontweight='bold',
           verticalalignment='top',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(calculate_field,
         frequency_khz=widgets.FloatSlider(min=20, max=60, step=1, value=40, 
                                          description='Frequency (kHz)'),
         ring_radius_lambda=widgets.FloatSlider(min=1.0, max=5.0, step=0.1, value=2.5,
                                               description='Ring Radius (λ)'),
         geometry_type=widgets.Dropdown(options=['fol', 'square', 'hexagonal'],
                                       value='fol',
                                       description='Geometry'),
         particle_size_mm=widgets.FloatSlider(min=1.0, max=8.0, step=0.5, value=3.0,
                                             description='Particle (mm)'));

## 🎯 Compare Geometries Side-by-Side

In [ ]:
def compare_geometries(frequency_khz, particle_size_mm):
    """
    Side-by-side comparison of FoL, Square, and Hexagonal
    """
    geometries = ['fol', 'square', 'hexagonal']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(f'Geometry Comparison @ {frequency_khz:.0f} kHz', 
                fontsize=16, fontweight='bold')
    
    for idx, geom in enumerate(geometries):
        # Calculate field (simplified version)
        # ... [calculation code similar to above]
        
        ax = axes[idx]
        # ... [plotting code]
        ax.set_title(geom.upper(), fontweight='bold')
    
    plt.tight_layout()
    plt.show()

interact(compare_geometries,
         frequency_khz=widgets.FloatSlider(min=20, max=60, step=2, value=40),
         particle_size_mm=widgets.FloatSlider(min=1, max=8, step=1, value=3));

## 📈 Performance Metrics

Calculate key performance indicators for different configurations

In [ ]:
def analyze_performance(emitters, frequency_khz):
    """
    Calculate and display performance metrics
    """
    # Calculate metrics for different geometries
    results = {}
    
    for geom in ['fol', 'square', 'hexagonal']:
        # ... calculations ...
        results[geom] = {'well_depth': 0, 'max_force': 0}  # Placeholder
    
    # Bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    
    geoms = list(results.keys())
    wells = [results[g]['well_depth'] for g in geoms]
    
    ax.bar(geoms, wells, color=['#2ecc71', '#3498db', '#e74c3c'], alpha=0.8)
    ax.set_ylabel('Well Depth (μJ)', fontweight='bold')
    ax.set_title(f'Performance Comparison ({emitters} emitters, {frequency_khz:.0f} kHz)',
                fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    plt.show()

interact(analyze_performance,
         emitters=widgets.Dropdown(options=[7, 19, 37], value=7),
         frequency_khz=widgets.FloatSlider(min=20, max=60, step=5, value=40));

## 💾 Export Data

Export results for external analysis

In [ ]:
def export_configuration(geometry, frequency_khz, filename):
    """
    Export emitter positions and field data to CSV
    """
    import pandas as pd
    
    # Generate positions
    # ... calculation ...
    
    # Save to CSV
    df = pd.DataFrame(positions, columns=['x', 'y', 'z'])
    df.to_csv(filename, index=False)
    
    print(f"✓ Exported to {filename}")

# Example
# export_configuration('fol', 40.0, 'fol_40khz_positions.csv')

## 🎓 Learn More

- [GitHub Repository](https://github.com/sportysport74/open-acoustic-levitation)
- [Theory Documentation](../theory/)
- [Build Guides](../builds/)

**Try adjusting the sliders above to explore the parameter space!**